# ===================================
# **WFLOW: DERIVING INPUT MAPS**
# ===================================

## ==== **PACKAGES** ====


In [1]:
using  Revise, Rasters, GeoFormatTypes, GeoTIFF, ArchGDAL, Shapefile, NCDatasets, GeoDataFrames
using Colors, ColorSchemes, CairoMakie, DimensionalData
using Geomorphometry

Path_Home = @__DIR__
cd(Path_Home)

include("Julia//Parameters.jl")
include("Julia//GeoRaster.jl")
include("Julia//GeoNetcdf.jl")
include("Julia//PlotParameter.jl")
include("Julia//GeoPlot.jl")
include("Julia//GeoLookuptable.jl")

# For new projects if does not exist
   mkpath(joinpath(Path_Root, Path_Forcing))
   mkpath(joinpath(Path_Root, Path_Gis))
   mkpath(joinpath(Path_Root, Path_Lookuptable))
   mkpath(joinpath(Path_Root, Path_NetCDF))
   mkpath(joinpath(Path_Root, Path_Julia))
   mkpath(joinpath(Path_Root, Path_Python))
   mkpath(joinpath(Path_Root, Path_TimeSeriesWflow))
   mkpath(joinpath(Path_Root, Path_Wflow))
   mkpath(joinpath(Path_Root, "Qgis"))
   mkpath(joinpath(Path_Root_NetCDF))


"D:\\JOE\\MAIN\\MODELS\\WFLOW\\Wflow.jl\\Wflow\\Data\\input\\Timoleague"

## **=== DEM OPERATIONS ===**

### **== DEM:** *Deriving data from Mosaic  ==*

In [ ]:
```
Deriving maps from mosaic
```
if 🎏_Mosaic
    Dem₀ = geoRaster.MOSAIC(;Path_Root_Mosaic, Missing=NaN)

    Path_Output_Mosaic = joinpath(Path_Root, Path_Gis, Filename_Input_Dem)

	# Reducing the size of the raw map with the shape file having the same shp file as MOSAIC
    Path_Input_Shp = joinpath(Path_Root, Path_Gis, Filename_Mask_Shp)
    Mask_Shapefile = GeoDataFrames.read(Path_Input_Shp)
	println(Path_Input_Shp)

	Dem_Resample₁ = Rasters.mask(Dem₀; with=Mask_Shapefile, missingval=NaN, shape=:polygon, boundary=:touches, progress=true)

	Rasters.write(Path_Output_Mosaic, Dem_Resample₁; ext=".tiff", missingval=NaN, force=true, verbose=true)
	println(Path_Output_Mosaic)
else
    printstyled("🎏_Mosaic=$🎏_Mosaic",color=:blue, italic=true )
end


### == **DEM:** *Resampling and deriving new CRS* ==

In [ ]:
"""	https://rafaqz.github.io/Rasters.jl/dev/api#Rasters.resample-Tuple
method: A Symbol or String specifying the method to use for resampling. From the docs for gdalwarp:

**:near:** nearest neighbour resampling (default, fastest algorithm, worst interpolation quality).
:bilinear: bilinear resampling.
:cubic: cubic resampling.
:cubicspline: cubic spline resampling.
:lanczos: Lanczos windowed sinc resampling.
:average: average resampling, computes the weighted average of all non-NODATA contributing pixels. rms root mean square / quadratic mean of all non-NODATA contributing pixels (GDAL >= 3.3)
:mode: mode resampling, selects the value which appears most often of all the sampled points.
:max: maximum resampling, selects the maximum value from all non-NODATA contributing pixels.
:min: minimum resampling, selects the minimum value from all non-NODATA contributing pixels.
:med: median resampling, selects the median value of all non-NODATA contributing pixels.
:q1: first quartile resampling, selects the first quartile value of all non-NODATA contributing pixels.
:q3: third quartile resampling, selects the third quartile value of all non-NODATA contributing pixels.
:sum: compute the weighted sum of all non-NODATA contributing pixels (since GDAL 3.1)"""

🎏_Method₁ = true
🎏_Method₂ = true

@assert(🎏_Method₁ || 🎏_Method₂)

# Reading data
	if 🎏_DemFromMosaic
        Path_Output_Mosaic = joinpath(Path_Root, Path_Gis, Filename_Input_Dem)
        Dem_Resample₀      = Rasters.Raster(Path_Output_Mosaic)
        Dem_Resample₁      = Rasters.replace_missing(Dem_Resample₀, missingval=NaN)

    elseif !(🎏_Mosaic)
        Path_Gis_Dem  = joinpath(Path_Root, Path_Gis, Filename_Input_Dem)
        Dem_Resample₀ = Rasters.Raster(Path_Gis_Dem)
        Dem_Resample₁ = Rasters.replace_missing(Dem_Resample₀, missingval=NaN)
	end

   # Resample Dem in 2 steps
      Param_Crs_GeoFormat = GeoFormatTypes.convert(WellKnownText, EPSG(Param_Crs))

    if 🎏_Method₁
        Dem_Resample₁ = Rasters.resample(Dem_Resample₁; res=Param_ΔX₁, method=Param_ResampleMethod₁, crs=Param_Crs_GeoFormat)
    end

    if 🎏_Method₂
      Dem_Resample₁  = Rasters.resample(Dem_Resample₁; res=Param_ΔX₂, method=Param_ResampleMethod₂, crs=Param_Crs_GeoFormat)
    end

    # Shiftlocus
        Dem_Resample = DimensionalData.shiftlocus(DimensionalData.Center(), Dem_Resample₁)

# Getting the metadata
	Metadatas = geoRaster.RASTER_METADATA(Dem_Resample; Verbose=true)

Dem_Resample₁ = Dem_Resample₀ = empty


In [ ]:
# import GeoInterface as GI
# Crs_Input = GI.crs(Dem_Resample)


### == **DEM:** *Longitute and Latitude* ==

In [ ]:
# Longitude, Latitude = Rasters.X(Metadatas.Coord_X_Left: Metadatas.ΔX: Metadatas.Coord_X_Right, crs=Metadatas.Crs_GeoFormat), Rasters.Y(Metadatas.Coord_Y_Top: Metadatas.ΔY: Metadatas.Coord_Y_Bottom, crs=Metadatas.Crs_GeoFormat)

Longitude = DimensionalData.X(LinRange(Metadatas.Coord_X_Left: Metadatas.ΔX: Metadatas.Coord_X_Right);  sampling=DimensionalData.Intervals(DimensionalData.Center()), crs=Metadatas.Crs_GeoFormat)

Latitude = DimensionalData.Y(LinRange(Metadatas.Coord_Y_Top: Metadatas.ΔY: Metadatas.Coord_Y_Bottom);  sampling=DimensionalData.Intervals(DimensionalData.Center()), crs=Metadatas.Crs_GeoFormat)

@assert(Metadatas.N_Width  == length(Longitude))
@assert(Metadatas.N_Height == length(Latitude))


### == **DEM:** *Deriving a coastal boundary* ==

In [ ]:
if 🎏_Coastline
	Coastline_Bool, Dem_Resample = geoRaster.DEM_DERIVE_COASTLINES(;Dem=Dem_Resample,  Longitude, Latitude, Crs=Metadatas.Crs_GeoFormat, Missing=NaN, DemMin=5.0)

	# Writting the output
	Path_Julia_Coastline = joinpath(Path_Root, Path_Julia, Filename_Coastline)
	Rasters.write(Path_Julia_Coastline, Coastline_Bool; ext=".tiff", missingval= NaN, force=true, verbose=true)

    if 🎏_Plots
	    using GLMakie
	    GLMakie.activate!()

	    geoPlot.HEATMAP(;🎏_Colorbar=true, Input=Coastline_Bool, Title="Digital Elevation Model", Label="Altitude [m]", colormap=:greys)
    end
end


### == **DEM:** *Reduce size of map by removing NaN* ==

In [ ]:
Dem_Resample = Rasters.trim(Dem_Resample)
Metadatas    = geoRaster.RASTER_METADATA(Dem_Resample; Verbose=true)

Longitude, Latitude = Rasters.X(Metadatas.Coord_X_Left: Metadatas.ΔX: Metadatas.Coord_X_Right, crs=Metadatas.Crs_GeoFormat), Rasters.Y(Metadatas.Coord_Y_Top: Metadatas.ΔY: Metadatas.Coord_Y_Bottom, crs=Metadatas.Crs_GeoFormat)

if 🎏_Plots
	using GLMakie
	GLMakie.activate!()

	geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Geomorphometry.multihillshade(Dem_Resample), Title="Digital Elevation Model", Label="Altitude [m]", colormap=:greys)
end


## == **WRITTING OUTPUTS** ==

In [ ]:
# Writting the output
	Path_Julia_Dem = joinpath(Path_Root, Path_Julia, Filename_Julia_Dem)
    println(Path_Julia_Dem)
	Rasters.write(Path_Julia_Dem, Dem_Resample; ext=".tiff", missingval= NaN, force=true, verbose=true)


## ====== **PLEASE RUN PYTHON CODE** =======

# === **DEM:** *Reading the corrected DEM from python & cleaning data* ===

In [ ]:
include("Julia//GeoRaster.jl")

# Reading data
   Path_InputPython_Dem = joinpath(Path_Root, Path_Python, Filename_Python_DemCorrected)

   Dem_Resample₀ = Rasters.Raster(Path_InputPython_Dem)
   Dem_Resample₁  = Rasters.replace_missing(Dem_Resample₀, missingval=NaN)

# Cleaning data
    Dem_Resample₂ = Rasters.trim(Dem_Resample₁)
    Dem_Resample = DimensionalData.shiftlocus(DimensionalData.Center(), Dem_Resample₂)

# Getting the metadata
    Metadatas    = geoRaster.RASTER_METADATA(Dem_Resample; Verbose=true)

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Geomorphometry.multihillshade(Dem_Resample), Title="Corrected Digital Elevation Model", Label="Altitude [m]", colormap=:greys)
end

# Writting the output
   Path_Julia_Dem_Corrected = joinpath(Path_Root, Path_Julia, Filename_Julia_DemCorrected)
   Rasters.write(Path_Julia_Dem_Corrected, Dem_Resample; ext=".tiff", missingval= NaN, force=true, verbose=true)

Dem_Resample₁ = Dem_Resample₀ = empty


## **== Longitude & Latitude ==**

In [ ]:

Longitude = DimensionalData.X(LinRange(Metadatas.Coord_X_Left: Metadatas.ΔX: Metadatas.Coord_X_Right);  sampling=DimensionalData.Intervals(DimensionalData.Center()), crs=Metadatas.Crs_GeoFormat)

Latitude = DimensionalData.Y(LinRange(Metadatas.Coord_Y_Top: Metadatas.ΔY: Metadatas.Coord_Y_Bottom);  sampling=DimensionalData.Intervals(DimensionalData.Center()), crs=Metadatas.Crs_GeoFormat)


## **== Subcatchment ==**

In [ ]:
if 🎏_MaskFromDem
    # Computing subcatchments
   Subcatchment = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat, missingval=0)

   Threads.@threads for iX=1:Metadatas.N_Width
      Threads.@threads for iY=1:Metadatas.N_Height
	    if !isnan(Dem_Resample[iX,iY])
            Subcatchment[iX,iY] = 1::Int64
         else
            Subcatchment[iX,iY] = 0::Int64
         end
      end # for iY=1:Metadatas.N_Height
   end # for iX=1:Metadatas.N_Width

else
    Path_InputPython_Subcatchment = joinpath(Path_Root, Path_Python, Filename_Python_CatchmentSubcatchment)
   #  Subcatchment₀                 = Rasters.Raster(Path_InputPython_Subcatchment)
   #  Subcatchment₀                 = Rasters.replace_missing(Subcatchment₀, missingval=0)

    # Computing subcatchments
    Subcatchment = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat)

    Threads.@threads for iX=1:Metadatas.N_Width
        Threads.@threads for iY=1:Metadatas.N_Height
            # if Subcatchment₀[iX,iY] == 1
            #  if Subcatchment₀[iX,iY] > 0
            if !isnan(Dem_Resample[iX,iY])
                Subcatchment[iX,iY] = 1::Int64
            else
                Subcatchment[iX,iY] = 0::Int64
            end
        end # for iY=1:Metadatas.N_Height
    end # for iX=1:Metadatas.N_Width
end # if 🎏_MaskFromDem

# Testing for issues
    🎏_Map₁_Eq_Map₂ = geoRaster.TEST_SAMESIZE(; Map₁=Dem_Resample, Map₂=Subcatchment, Map₁_Nodata=NaN, Map₂_Nodata=0)

# Writting the output
   Path_Subcatchment = joinpath(Path_Root, Path_Wflow, Filename_Wflow_Subcatchment)
   println(Path_Subcatchment)
   Rasters.write(Path_Subcatchment, Subcatchment; ext=".tiff", missingval= NaN, force=true, verbose=true)

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input=Subcatchment, Title="Subcatchment", Label="Subcatchment")
end


## === **CATCHMENT:** *Reading catchment boundary from Python used as a mask* ===

## **DEM:** *Mask to catchment area*

In [ ]:
# include("Julia//GeoRaster.jl")

# Dem_Resample_Mask = geoRaster.MASK(;Input=Dem_Resample, Mask=Subcatchment, Param_Crs=Metadatas.Crs_GeoFormat, Latitude, Longitude)

# 🎏_Map₁_Eq_Map₂ = geoRaster.TEST_SAMESIZE(; Map₁=Dem_Resample_Mask, Map₂=Subcatchment, Map₁_Nodata=NaN, Map₂_Nodata=0)

# # Writting output
#    Path_Julia_DemMask = joinpath(Path_Root, Path_Julia, Filename_Julia_Dem_Mask)

#    Rasters.write(Path_Julia_DemMask, Dem_Resample_Mask; ext=".tiff", missingval= NaN, force=true, verbose=true)

# # Getting the metadata
# 	Metadatas = geoRaster.RASTER_METADATA(Dem_Resample_Mask; Verbose=true);

# if 🎏_Plots
# 	geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Dem_Resample_Mask, Title="Digital Elevation Model Masked", Label="Altitude [m]")
# end


## == **DEM:** *flow accumulation* ==

In [ ]:
if 🎏_Plots && 🎏_Plot_FlowAccumulation
   # Options: DInf(), D8(), FD8(2)
   Acc, Ldd = Geomorphometry.flowaccumulation(Dem_Resample; method=FD8(4))

   Acc_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=Acc, Latitude, Longitude, Mask=Subcatchment)

	geoPlot.HEATMAP(;🎏_Colorbar=true, Input= log10.(Acc_Mask), Title="Flow accumulation", Label="Flow accumulation [mm]", colormap=:rain)
end


## == **LDD:** *Python Mask to catchment area* == 

In [ ]:
include("Julia//GeoRaster.jl")

Path_InputPython_Ldd = joinpath(Path_Root, Path_Python, Filename_Python_Ldd)
Ldd₁                 = Rasters.Raster(Path_InputPython_Ldd)
Ldd₁                 = Rasters.replace_missing(Ldd₁, missingval=0)

# Masking with catchment
   Ldd_Mask = Rasters.resample(Ldd₁; to=Dem_Resample, missingval=0)

   Ldd_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=Ldd_Mask, Latitude, Longitude, Mask=Subcatchment, Missing=0)

# Writting output
   Path_Wflow_LddMask = joinpath(Path_Root, Path_Wflow, Filename_Wflow_Ldd)
   Rasters.write(Path_Wflow_LddMask, Ldd_Mask; ext=".tiff", force=true, verbose=true)
   println(Path_Wflow_LddMask)

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input=Ldd_Mask, Title="Local Drain Direction", Label="Ldd [1-9]", colormap=Makie.Categorical(:Spectral_11))
end

🎏_Map₁_Eq_Map₂ = geoRaster.TEST_SAMESIZE(; Map₁=Subcatchment, Map₂=Ldd_Mask, Map₁_Nodata=0, Map₂_Nodata=0)


## == **GAUGE** ==

In [ ]:
""" Sometimes the Near() method returns the right pixel data, but it’s strange, it “fails” with coordinate points that are pretty at the center of their pixel..
 But also note your Raster does not contain “pixels” that have a center (Intervals on x/y), but Points. So likely your confusion is from treating them as pixels starting from the point at one side"""

Path_OutputGauge = joinpath(Path_Root, Path_Wflow, Filename_Wflow_Gauge)

Gauge, iiParam_GaugeCoordinate = geoRaster.GAUGE(;🎏_Method_Index = "Rasters", Latitude=Latitude, Longitude=Longitude, Metadatas=Metadatas, Param_GaugeCoordinate, Path_OutputGauge)

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Gauge, Title="Gauge", Label="Altitude [m]", colormap=:reds)
end

println(Path_OutputGauge)


## ==**RIVER:** *Mask to catchment area*==

In [ ]:
include("Julia//Parameters.jl")

if 🎏_RiverFromDem
    Path_Input_RiverFromDem = joinpath(Path_Root, Path_Python, Filename_Python_Dem2Rivers)
    println(Path_Input_RiverFromDem)
    River₀ = Rasters.Raster(Path_Input_RiverFromDem)

    River = Rasters.resample(River₀; to=Dem_Resample, missingval=0)

    for iX=1:Metadatas.N_Width
        for iY=1:Metadatas.N_Height
            if !(ismissing(River[iX,iY]))
                if River[iX,iY] > 0
                    River[iX,iY] = 1
                else
                    River[iX,iY] = 0
                end
            else
                River[iX,iY] = 0
            end
        end # for iY=1:Metadatas.N_Height
    end # for iX=1:Metadatas.N_Width

else
    Path_Gis_River  = joinpath(Path_Root_Data, Path_River, Filename_River_Shp)
    println(Path_Gis_River)
    @assert isfile(Path_Gis_River)

    # Load the shapefile
    River_Raw = Shapefile.Handle(Path_Gis_River)

    River = Rasters.rasterize(minimum, River_Raw; res=Param_ΔX₂, fill=1, missingval=0, crs=Metadatas.Crs_GeoFormat, to=Dem_Resample, shape=:point ,progress=true, boundary=:center)
end # if 🎏_RiverFromDem

River = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=River, Latitude, Longitude, Mask=Subcatchment, Missing=0)

Path_Wflow_River = joinpath(Path_Root, Path_Wflow, Filename_Wflow_Rivers)
Rasters.write(Path_Wflow_River, River; ext=".tiff", force=true, verbose=true, missingval=0)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_6 =  CairoMakie.Figure()
   Axis_6 = CairoMakie.Axis(Fig_6[1, 1], title="Rivers", xlabel= L"$Latitude$", ylabel=L"$Longitude$", ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

   Map_6 = CairoMakie.heatmap!(Axis_6, River, colormap=Reverse(:lajolla))
#    Map_6 = CairoMakie.heatmap!(Axis_6, Gauge, colormap=:reds)

#    CairoMakie.Colorbar(Fig_6[1, 2], Map_6, label = "Rivers", width = 15, ticksize = 15, tickalign = 0.5)
   CairoMakie.display(Fig_6)
end


## ==**RIVER ORDER:** *Mask to catchment area*==

In [ ]:
include("Julia//Parameters.jl")

Path_Input_RiverOrder = joinpath(Path_Root, Path_Python, Filename_Python_RiverOrder)
println(Path_Input_RiverOrder)
RiverOrder₀ = Rasters.Raster(Path_Input_RiverOrder)

RiverOrder = Rasters.resample(RiverOrder₀; to=Dem_Resample, missingval=0)

for iX=1:Metadatas.N_Width
   for iY=1:Metadatas.N_Height
      if !(ismissing(RiverOrder[iX,iY]))
            if RiverOrder[iX,iY] > 0
               RiverOrder[iX,iY] = RiverOrder[iX,iY]
            else
               RiverOrder[iX,iY] = 0
            end
      else
            RiverOrder[iX,iY] = 0
      end
   end # for iY=1:Metadatas.N_Height
end # for iX=1:Metadatas.N_Width

RiverOrder = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=RiverOrder, Latitude, Longitude, Mask=Subcatchment, Missing=0)

Path_Output_RiverOrder = joinpath(Path_Root, Path_Julia, Filename_Julia_RiverOrder)
Rasters.write(Path_Output_RiverOrder, RiverOrder; ext=".tiff", force=true, verbose=true, missingval=0)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_6 =  CairoMakie.Figure()
   Axis_6 = CairoMakie.Axis(Fig_6[1, 1], title="River Order", xlabel= L"$Latitude$", ylabel=L"$Longitude$", ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

   Map_6 = CairoMakie.heatmap!(Axis_6, RiverOrder, colormap=Reverse(:lajolla))

   CairoMakie.display(Fig_6)
end


## == **PITS**==

In [ ]:
#   Ldd_Mask[iiParam_GaugeCoordinate[1],iiParam_GaugeCoordinate[2]] = 5

Pits  = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat,  missingval=0)

Pits_Count = 0
for iX=1:Metadatas.N_Width
   for iY=1:Metadatas.N_Height
      if Ldd_Mask[iX, iY] == 5
         Pits_Count += 1
         Pits[iX,iY] = Pits_Count
         display("[$iX, $iY]")
    else
        Pits[iX,iY] = NaN
      end
   end # for iY=1:Metadatas.N_Height
end # for iX=1:Metadatas.N_Width

println("	Pits = $Pits_Count")

# Writting output
   PathOutput_Pits = joinpath(Path_Root, Path_Julia, Filename_Julia_Pits)
   Rasters.write(PathOutput_Pits, Pits; ext=".tiff", force=true, verbose=true)
   println(PathOutput_Pits)


## == **SLOPE:** *Reading Slope from Python & Mask* ==

In [ ]:
include("Julia//GeoRaster.jl")

Path_InputPython_Slope = joinpath(Path_Root, Path_Python, Filename_Python_Slope)

Slope₀ = Rasters.Raster(Path_InputPython_Slope)

Slope = Rasters.replace_missing(Slope₀, missingval=NaN)

Slope = Rasters.resample(Slope₀; to=Dem_Resample, missingval=0)

Slope_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=Slope, Latitude, Longitude, Mask=Subcatchment)

# Slope_Mask = Rasters.replace_missing(Slope₀, missingval=NaN)

# Writting the output
Path_Wflow_Slope = joinpath(Path_Root, Path_Wflow, Filename_Wflow_Slope)
Rasters.write(Path_Wflow_Slope, Slope_Mask; ext=".tiff", force=true, verbose=true)
println(Path_Wflow_Slope)

if 🎏_Plots
	geoPlot.HEATMAP(;🎏_Colorbar=true, Input=Slope_Mask, Title="Slope", Label="Slope", colormap=:matter)
end

# 🎏_Map₁_Eq_Map₂ = geoRaster.TEST_SAMESIZE(; Map₁=Subcatchment, Map₂=Slope_Mask,  Map₁_Nodata=NaN, Map₂_Nodata=NaN)

Slope₀ = nothing


## == **Rivers slope** ==

In [ ]:
include("Julia//GeoRaster.jl")

RiverSlope = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=Slope_Mask, Latitude, Longitude, Mask=River)

Path_Wflow_RiverSlope = joinpath(Path_Root, Path_Wflow, Filename_Wflow_RiverSlope)

Rasters.write(Path_Wflow_RiverSlope, RiverSlope; ext=".tiff", force=true, verbose=true)
println(Path_Wflow_RiverSlope)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_12 =  CairoMakie.Figure()
   Axis_12 = CairoMakie.Axis(Fig_12[1, 1], title="River Slope Per Pixel", xlabel= L"$Latitude$", ylabel=L"$Longitude$", ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize, titlecolor=titlecolor)

   # CairoMakie.plot!(Axis_12, River, colormap=Reverse(:lajolla))
   Map_12 = CairoMakie.heatmap!(Axis_12, RiverSlope, colormap=:viridis)

   Colorbar(Fig_12[1, 2], Map_12, label = "River slope [deg]", width = 15, ticksize = 15, tickalign = 0.5)
   display(Fig_12)
end


## == **River width** ==

In [ ]:
# RiverWidth = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat, mappedcrs=EPSG(Metadatas.Param_Crs))

# for iX=1:Metadatas.N_Width
#    for iY=1:Metadatas.N_Height
#       if River[iX,iY] == 1
#          RiverWidth[iX,iY] = Param_RiverWidth
#       else
#          RiverWidth[iX,iY] = NaN
#       end
#    end # for iY=1:Metadatas.N_Height
# end # for iX=1:Metadatas.N_Width

# Path_Wflow_RiverWidth = joinpath(Path_Root, Path_Wflow, Filename_Wflow_RiverWidth)
# # Rasters.write(Path_Wflow_RiverWidth, RiverWidth; ext=".tiff", force=true, verbose=true)

# if 🎏_Plots
#    CairoMakie.activate!()
#    Fig_9 =  CairoMakie.Figure()
#    Axis_9 = CairoMakie.Axis(Fig_9[1, 1], title="River Width", xlabel= L"$Latitude$", ylabel=L"$Longitude$",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

#    Map_9 = CairoMakie.plot!(Axis_9, RiverWidth, colormap=:lajolla)

#    # CairoMakie.Colorbar(Fig_6[1, 2], Map_6, label = "Rivers", width = 15, ticksize = 15, tickalign = 0.5)
#    CairoMakie.display(Fig_9)
# end


## == **River depth** ==

In [ ]:
# RiverDepth = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat)

# for iX=1:Metadatas.N_Width
#    for iY=1:Metadatas.N_Height
#       if River[iX,iY] == 1
#          RiverDepth[iX,iY] = Param_RiverDepth
#       else
#          RiverDepth[iX,iY] = NaN
#       end
#    end # for iY=1:Metadatas.N_Height
# end # for iX=1:Metadatas.N_Width

# Path_Wflow_RiverDepth = joinpath(Path_Root, Path_Wflow, Filename_Wflow_RiverDepth)
# println(Path_Wflow_RiverDepth)

# Rasters.write(Path_Wflow_RiverDepth, RiverDepth; ext=".tiff", force=true, verbose=true, crs=Metadatas.Crs_GeoFormat)

# if 🎏_Plots
#    CairoMakie.activate!()
#    Fig_10 =  CairoMakie.Figure()
#    Axis_10 = CairoMakie.Axis(Fig_10[1, 1], title="River Depth", xlabel= L"$Latitude$", ylabel=L"$Longitude$",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

#    Map_10 = CairoMakie.plot!(Axis_10, RiverDepth, colormap=Reverse(:viridis))

#    # CairoMakie.Colorbar(Fig_6[1, 2], Map_6, label = "Rivers", width = 15, ticksize = 15, tickalign = 0.5)
#    CairoMakie.display(Fig_10)
# end


## == **River length** ==

In [ ]:
Path_InputPython_RiverLength = joinpath(Path_Root, Path_Python, Filename_Python_RiverLength)
println(Path_InputPython_RiverLength)

RiverLength₀ = Rasters.Raster(Path_InputPython_RiverLength)
RiverLength₁ = Rasters.resample(RiverLength₀; to=Dem_Resample, missingval=0)

RiverLength₁ = Rasters.replace_missing(RiverLength₁, missingval=NaN)
RiverLength_Mask = geoRaster.MASK(;Param_Crs=Metadatas.Crs_GeoFormat, Input=RiverLength₁, Latitude, Longitude, Mask=River)

Path_Wflow_RiverLength = joinpath(Path_Root, Path_Wflow, Filename_Wflow_RiverLength)
Rasters.write(Path_Wflow_RiverLength, RiverLength_Mask; ext=".tiff", force=true, verbose=true)
println(Path_Wflow_RiverLength)

if 🎏_Plots
   CairoMakie.activate!()
   Fig_12 =  CairoMakie.Figure()
   Axis_12 = CairoMakie.Axis(Fig_12[1, 1], title="River Length Per Pixel", xlabel= L"$Latitude$", ylabel=L"$Longitudegitude$",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize, titlecolor=titlecolor)

   Map_12 = CairoMakie.heatmap!(Axis_12, RiverLength_Mask, colormap=:viridis)

   Colorbar(Fig_12[1, 2], Map_12, label = "River length [m]", width = 15, ticksize = 15, tickalign = 0.5)
   display(Fig_12)
end


## == **River Manning** ==


In [ ]:
# RiverManning = Rasters.Raster((Longitude, Latitude), crs=Metadatas.Crs_GeoFormat)

# for iX=1:Metadatas.N_Width
#    for iY=1:Metadatas.N_Height
#       if River[iX,iY] == 1
#          RiverManning[iX,iY] = Param_RiverManning
#       else
#          RiverManning[iX,iY] = NaN
#       end
#    end # for iY=1:Metadatas.N_Height
# end # for iX=1:Metadatas.N_Width

# Path_Wflow_RiverManning = joinpath(Path_Root, Path_Wflow, Filename_Wflow_RiverManning)
# println(Path_Wflow_RiverManning)

# Rasters.write(Path_Wflow_RiverManning, RiverManning; ext=".tiff", force=true, verbose=true, crs=Metadatas.Crs_GeoFormat)

# if 🎏_Plots
#    CairoMakie.activate!()
#    Fig_10 =  CairoMakie.Figure()
#    Axis_10 = CairoMakie.Axis(Fig_10[1, 1], title="River Manning", xlabel= L"$Latitude$", ylabel=L"$Longitude$",  ylabelsize=ylabelsize, xlabelsize=xlabelSize, xticksize=xticksize, yticksize=yticksize, titlesize=titlesize,  titlecolor=titlecolor)

#    Map_10 = CairoMakie.plot!(Axis_10, RiverManning, colormap=Reverse(:viridis))

#    # CairoMakie.Colorbar(Fig_6[1, 2], Map_6, label = "Rivers", width = 15, ticksize = 15, tickalign = 0.5)
#    CairoMakie.display(Fig_10)
# end


# === **LOOKUP TABLES**===


## == **SoilMap:** *LookupTable* ==

In [ ]:
if 🎏_SoilMap
    include("Julia//Parameters.jl")
	 include("Julia//GeoLookuptable.jl")

   Path_Home = @__DIR__
	Path₀ = abspath(joinpath(Path_Home, ".."))
 	Path_Lookup_Hydro = joinpath(Path₀, Path_Root_LookupTable, Filename_Lookuptable_Hydro)
   @assert isfile(Path_Lookup_Hydro)

   Path_Input_Map = joinpath(Path_Root, Path_Gis, Filename_SoilMap_Shp)

   Soil_Header, Soil_Maps = geoLookuptable.LOOKUPTABLE_2_MAPS(;🎏_Plots, Dem_Resample, Path_Input_Map, Latitude, Longitude, Path_Lookup=Path_Lookup_Hydro, ShpLayer=Layer_Soil, Metadatas, Param_Crs=Metadatas.Crs_GeoFormat, Path_Root, Subcatchment, TitleMap="Soil Map", ΔX=Param_ΔX₂)
else
   Soil_Header = Soil_Maps = []
end # if 🎏_SoilMap



## == **LandUse:** *LookupTable* ==

In [ ]:
include("Julia//GeoLookuptable.jl")
include("Julia//Parameters.jl")
include("Julia//GeoRaster.jl")
if 🎏_LandUseMap

   Path_Home = @__DIR__
	Path₀ = abspath(joinpath(Path_Home, ".."))
 	Path_Lookup_LandUse = joinpath(Path₀, Path_Root_LookupTable, Filename_Lookuptable_LandUse)
   @assert isfile(Path_Lookup_LandUse)

   Path_Input_Map = joinpath(Path_Root, Path_Gis, Filename_LandUseMap_Shp)

   LandUse_Header, LandUse_Maps = geoLookuptable.LOOKUPTABLE_2_MAPS(;🎏_Plots, Dem_Resample, Path_Input_Map, Latitude, Longitude, Path_Lookup=Path_Lookup_LandUse, ShpLayer=Layer_LandUse, Metadatas, Param_Crs=Metadatas.Crs_GeoFormat, Path_Root, Subcatchment, TitleMap="LandUse Map", ΔX=Param_ΔX₂)
else
   LandUse_Header = LandUse_Maps = []

end # if 🎏_LandUseMap


## == **River:** *Lookup table* ==

In [ ]:
include("Julia//GeoLookuptable.jl")
include("Julia//Parameters.jl")
include("Julia//GeoRaster.jl")

Path_Input_Map = joinpath(Path_Root, Path_Julia, Filename_Julia_RiverOrder)
Path_Lookup_River = joinpath(Path_Root, Path_Lookuptable, Filename_Lookuptable_River)

River_Header, River_Maps = geoLookuptable.LOOKUPTABLE_2_MAPS(;🎏_Plots=false, Dem_Resample, Path_Input_Map, Latitude, Longitude, Path_Lookup=Path_Lookup_River, ShpLayer=Layer_LandUse, Metadatas, Param_Crs=Metadatas.Crs_GeoFormat, Path_Root, Subcatchment, TitleMap="LandUse Map", ΔX=Param_ΔX₂)


## ==**Impermeable layers:** *LookupTable*==

# =**NETCDF**=

## ==  ***Static maps*** ==

In [ ]:
include("Julia//Parameters.jl")
include("Julia//GeoNetcdf.jl")

if 🎏_NetCDF
   NetCDF, Path_NetCDF_Full = geoNetcdf.TIFF_2_NETCDF(Filename_Wflow_RiverManning, Filename_Wflow_RiverDepth, Filename_Wflow_Rivers, Filename_Wflow_RiverSlope, Filename_Wflow_RiverWidth, Filename_Wflow_Subcatchment, Gauge, Latitude, Ldd_Mask, Longitude, Metadatas, River, RiverDepth, RiverLength_Mask, RiverManning, RiverSlope, RiverWidth, Slope_Mask, Soil_Header, Soil_Maps, Subcatchment, LandUse_Header, LandUse_Maps)
end

# SAVING FILE
   # Path_Copy =  joinpath(Path_Root, Path_NetCDF, Filename_NetCDF_Instates)
   # Path_Paste  = joinpath(Path_Root_NetCDF, Filename_NetCDF_Instates)
   # Base.Filesystem.cp(Path_Copy, Path_Paste, force=true)
   # geoNetcdf.COPYPASTE_LARGE_FILES(;Path_Copy, Path_Paste)


In [ ]:
if 🎏_NetCDF
	Output_NCDatasets = NCDatasets.NCDataset(Path_NetCDF_Full)
	close(Output_NCDatasets)
end


### ***Plotting NetCDF maps***

In [ ]:
if 🎏_Plots && 🎏_Plot_NetCDF
	using NCDatasets

	Path_NetCDF_Full  = joinpath(Path_Root, Path_NetCDF, Filename_NetCDF_Instates)
	Output_NCDatasets = NCDatasets.NCDataset(Path_NetCDF_Full)

	Keys = NCDatasets.keys(Output_NCDatasets)

	Keys_Select = []
	for iiKeys ∈ Keys
		Output = Output_NCDatasets[iiKeys]
		Dimensions = length(size(Output))

		if Dimensions == 2
				push!(Keys_Select, String(iiKeys))
		end # Dimensions == 2
	end # for iiKeys ∈ Keys

	Keys_Select = String.(Keys_Select)
	println(Keys_Select)

	for iiKeys ∈ Keys_Select
		println(iiKeys)
		Map₀ = Array(Output_NCDatasets[iiKeys])

		# Clean data
			FillValue = Output_NCDatasets[iiKeys].attrib["_FillValue"]
			Map =  replace(Map₀, FillValue => NaN)

		geoPlot.HEATMAP(;🎏_Colorbar=true, Input=Map, Label="$iiKeys", Title="$iiKeys", Xlabel= L"$iX$", Ylabel=L"$iY$", Yreversed=true, colormap=:hawaii50)
	end
end  # if: 🎏_Plots &&

Output_NCDatasets = nothing


## ***Timeseries NetCDF maps***

In [ ]:
include("Julia//Parameters.jl")

if 🎏_Forcing_2_NetCDF
	include("Julia//GeoRaster.jl")
	include("Julia//GeoNetcdf.jl")

	NetCDFmeteo, Path_NetCDFmeteo_Output = geoNetcdf.TIMESERIES_2_NETCDF(Latitude, Longitude, Metadatas, Subcatchment; NsplitYear=3)

		# NetCDFmeteo, Path_NetCDFmeteo_Output = geoNetcdf.TIMESERIES_2_NETCDF(Latitude, Longitude, Metadatas, Subcatchment;  )

	if 🎏_Plot_TimeSeries
   	# precip pet temp
   	geoPlot.HEATMAP_TIME(;Path=Path_NetCDFmeteo_Output, NameOutput="temp", Layer=1)
	end

   # Saving NetCDF
   #    Path_Copy = joinpath(Path_Root, Path_TimeSeriesWflow, Filename_NetCDF_Forcing)

   # Path_Paste  = joinpath(Path_Root_NetCDF, Filename_NetCDF_Forcing)
   # Base.Filesystem.cp(Path_Copy, Path_Paste, force=true)

end
